In [99]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
from my_library import inverse_modulo
from Crypto.Cipher import AES

a = 0x1337deadbeef
b = 0xb
m = 2**48


def next_state(state):
    state = (a * state + b) % m
    return state
    

data = {"msg": "Try encrypting this plaintext with the given IV and my secret key!", "plaintext": "d57d5ca042895dc4c417e91d22515586", "IV": "4edafc3d6b55d5789af976a56c54f1f6"}
player = bytes.fromhex(data["plaintext"][16:])
t_array = []
for i in range(1, len(player)):
    t_array.append(a ** i % m)
a_array = []
for i in range(1, len(player)):
    s = 0
    for j in range(i):
        s = (s + a ** j) % m
    s = s * b % m
    s = (player[i] * 2 ** 40 - a ** i * player[0] * 2 ** 40 - s) % m
    a_array.append(s)
A = Matrix(ZZ, 9, 9)
for i in range(7):
    A[i, i] = m
for i in range(7):
    A[7, i] = t_array[i]
for i in range(7):
    A[8, i] = a_array[i]
A[8, 8] = 2 ** 40
A2 = A.LLL()
r_array = []
for i in range(7):
    r_array.append((0 - A2[5, i]) % (2 ** 40))
s_array = []
s_array.append(((r_array[0] + a_array[0]) * inverse_modulo(t_array[0], m) % m + player[0] * 2 ** 40) % m)
for i in range(1, 8):
    s_array.append(next_state(s_array[i - 1]))
state = s_array[7]
key = b""
for i in range(8):
    state = next_state(state)
    add = state >> 40
    key += long_to_bytes(add)

player = bytes.fromhex(data["IV"][:16])
t_array = []
for i in range(1, len(player)):
    t_array.append(a ** i % m)
a_array = []
for i in range(1, len(player)):
    s = 0
    for j in range(i):
        s = (s + a ** j) % m
    s = s * b % m
    s = (player[i] * 2 ** 40 - a ** i * player[0] * 2 ** 40 - s) % m
    a_array.append(s)
A = Matrix(ZZ, 9, 9)
for i in range(7):
    A[i, i] = m
for i in range(7):
    A[7, i] = t_array[i]
for i in range(7):
    A[8, i] = a_array[i]
A[8, 8] = 2 ** 40
A2 = A.LLL()
r_array = []
for i in range(7):
    r_array.append((0 - A2[4, i]) % (2 ** 40))
s_array = []
s_array.append(((r_array[0] + a_array[0]) * inverse_modulo(t_array[0], m) % m + player[0] * 2 ** 40) % m)
for i in range(1, 8):
    s_array.append(next_state(s_array[i - 1]))
state = s_array[0]
key2 = b""
for i in range(8):
    state = (state - b) * inverse_modulo(a, m) % m
    add = state >> 40
    key2 = long_to_bytes(add) + key2
key = key + key2
cipher = AES.new(key, AES.MODE_CBC, bytes.fromhex(data["IV"]))
print(cipher.encrypt(bytes.fromhex(data["plaintext"])).hex())

2824399ec8fdc9ec
4037af1d93305cc8af945aa56003c779
